# **Ex1: Linear Regression and methods for finding the minimum**

---

Elad David (206760274) & Inbar Shmaya (208774026)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

# Stop condition variables:
epsilon = 0.0000001  # 10 ^ -8
delta = 0.0001 # 10 ^ -5
max_iterations = 150

# **Question 1:** Load data file and initialize matrix X and vector y

In [2]:
f = np.loadtxt(open("/content/drive/MyDrive/ML/EX1/data.csv", 'r'), delimiter = ",") # Load the data from the file

numOfRows = len(f)  
numOfCols = len(f[0]) 

# Padd with 1's vector
oneVector = np.ones((numOfRows, 1)) # For theta_0 variable
for i in oneVector: # np.ones becomes a vector of NANs in the next code scope
  i = 1
f = np.column_stack((oneVector, f))
data = f.T # Transpose the matrix-data in the file

x = data[0 : numOfCols - 1]
y = data[numOfCols -1 : numOfCols][0]
m = len(x[0]) # num of samples
n = len(x) - 1 # num of features (not including the 1's column)

NameError: ignored

# **Section A:** Normalize the data (using the normal-distribution method)

In [ ]:
def normalize_col(v):
  """
  Return normalized column using the normal-distribution method
  """
  ave = np.average(v)  # Calculate average
  std = np.nanstd(v)   # Calculate Standard-deviation
    
  # Avoid "division in 0" error
  # If std = 0 --> There is no need to normalize (no changes in the variables) 
  if std == 0: # Happens when all the values in the column are equal (like the 1's column)
    print("This case is the 1's column:")
    print("Average after the normalization:            1")
    print("Standard-deviation after the normalization: 0")
    print()
    return v
    
  for num in range(len(v)):  # Normalize each cell in the vector  
    v[num] = (v[num] - ave) / std


  # Check if failed
  ave_check = np.average(v)
  std_check = np.std(v)
  assert (0 - epsilon < ave_check < 0 + epsilon) and (1 - epsilon < std_check < 1 + epsilon), "Error: Normalizing the vector has been failed"
  
  print("Average after the normalization:            ", int(float('{0:.1f}'.format(ave_check))))
  print("Standard-deviation after the normalization: ", int(float(std_check)))
  print()
  
  return v
  

# Show that the ave is 0 and the std is 1 
print("Show that the average equals 0 and the standard-deviation equals 1 for each column of the data:\n")

for col in x:
  col = normalize_col(col)
  

y = normalize_col(y)
print("Average of y after the normalization:            " , int(float('{0:.1f}'.format(np.average(y)))))
print("Standard-deviation of y after the normalization: " , int(float(np.nanstd(y))))
print()

# **Section B:** Calculate h_theta(x) in linear-regression case

In [ ]:
x = x.T # Transpose for the multiplication
def h_theta(theta_v, x):
  return x.dot(theta_v) 

# **Section C:** Calculate J(theta), aka price function

In [ ]:
def calc_j(theta_v, x, y):
  sum = 0
  for i in range(m): # Sum for all rows/ samples
    sum += pow(h_theta(theta_v, x[i]) - y[i], 2) # (h(x[i]) - y[i])^2
  return (1/(2*m)) * sum

# **Section D:** Calculate grad_J(theta)

In [ ]:
def calc_grad_j(theta_v, x, y):
  grad_j = h_theta(theta_v, x) 
  # Calculate grad j - y
  for i in range(len(y)):
    grad_j[i] -= y[i] # h(x) - y
  grad_j = np.dot(x.T, grad_j) / m # x.T * (h(x) - y) / m
  
  return grad_j 

# **Section E:** Gradient-Descent

In [ ]:
def Gradient_Descent(alpha):
  theta_v = np.ones((n + 1, 1)) / 3 # Initialize temp theta to (1/3 ... 1/3) (n+1*1)
  j_theta = [] # Using for saving all the errors of each iteration (for plotting them later)
  j_theta.append(calc_j(theta_v, x, y)) # Calculate j of init theta
  iter = 0
  
  while iter < max_iterations:
    # Calculate next theta vector
    next_theta = theta_v - (alpha * calc_grad_j(theta_v, x, y))
    
    # Stop conditions:
    if np.linalg.norm(np.subtract(next_theta, theta_v)) < epsilon: # Norm the thetas vector
      break
    if abs(calc_j(next_theta, x, y) - calc_j(theta_v, x, y)) < delta: # Error (J) delta
      break
    
    theta_v = next_theta
    j_theta.append(calc_j(theta_v, x, y)) # Update the error of the calculation
    iter += 1
  
  # Figure:
  fig = plt.figure()
  plt.plot(j_theta)
  plt.suptitle("Gradient Descent: J(theta) graph")
  plt.title("alpha= " + str(alpha))
  plt.xlabel("Iterations")
  plt.ylabel("J(theta)")
  plt.show()

alpha = 1
for i in range(3):
  alpha /= 10
  Gradient_Descent(alpha)
  print()


alpha = 0.1 


**Analysing the results - Optimal alpha:**

The plots above have a negative slope, meaning the J(theta) is lower for every iteration added.

As you can see, the first graph (when alpha equals 0.1) is convergent to 0 the fastest.

On the X-axis the number of iterations is represented, and comparing the number of iterations on each graph, the first graph is still the fastest to be convergented.

For conclusion, we should choose the next alpha value (according to the tests above):

Alpha = 0.1 מתכנס באופן מהיר ויעיל


# **Section F:** Stochastic GD algorithm

In [ ]:
def stochastic_GD():
  alpha = 0.05
  theta_v = np.ones((n + 1, 1)) / 3 # Init temp theta to (1/3 ... 1/3) (n+1*1)
  j_theta = []
  j_theta.append(calc_j(theta_v, x, y)) # Calculate j of init theta
  iter = 0
  k = 0
  i = 1

  while iter < max_iterations:
    # Calculate new theta considering 1 examle 
    x_i = x[i : i+1] 
    y_i = y[i : i+1] 
    theta_v = theta_v - (alpha * calc_grad_j(theta_v, x_i, y_i))
    
    j_theta.append(calc_j(theta_v, x, y))
    k += 1
    i += 1
    iter += 1

  # Figure:
  fig = plt.figure()
  plt.plot(j_theta)
  plt.suptitle("stochastic-GD: J(theta) graph")
  plt.title("alpha= " + str(alpha))
  plt.xlabel("Iterations")
  plt.ylabel("J(theta)")
  plt.show()

stochastic_GD()

# **Section F:** Mini-Batch algorithm

In [ ]:
def mini_batch():
  theta_v = np.ones((n + 1, 1)) / 3 # Init temp theta to (1/3 ... 1/3) (n+1*1)
  j_theta = []
  j_theta.append(calc_j(theta_v, x, y)) # Calculate j of init theta
  T = 30 # Num of groups (~100 for each batch; 3047 samples total)
  N = int(m/T) # Group size
  iter = 0
  k = 0

  while iter < max_iterations:
    i_from = (k * N) % m
    i_to = ((k + 1) * N - 1) % m
    x_i = x[i_from : i_to] # slice just rows
    y_i = y[i_from : i_to]
    next_theta = theta_v - (alpha * calc_grad_j(theta_v, x_i, y_i))
    
    # Stop conditions:
    if np.linalg.norm(np.subtract(next_theta, theta_v)) < epsilon: 
      break
    if abs(calc_j(next_theta, x, y) - calc_j(theta_v, x, y)) < delta:
      break
    
    theta_v = next_theta
    j_theta.append(calc_j(theta_v, x, y))
    k += 1
    
    
    iter += 1

  # Figure:
  fig = plt.figure()
  plt.plot(j_theta)
  plt.suptitle("Mini-Batch: J(theta) graph")
  plt.title("alpha= " + str(alpha))
  plt.xlabel("Iterations")
  plt.ylabel("J(theta)")
  plt.show()

mini_batch()


# **Section F (cont.):** Analyse run-time

In [ ]:
def time_function(f, *args):
  """
  Call a function f with args and return the time (in seconds) that it took to execute.
  """
  import time
  tic = time.time()
  f(*args)
  toc = time.time()
  return toc - tic

s_time = time_function(stochastic_GD,)
print('Stochastic GD algorithm took %f seconds' % s_time)

print()

m_b_time = time_function(mini_batch,)
print('Mini-Batch algorithm took %f seconds' % m_b_time)
